# Simple tool to read the demo JSON data model file

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import requests

Note: Requires Pandas version +1.0

### Load the JSON file

In [ ]:
try:
    # first try to load the file from your computer
    cwd = os.getcwd()
    file_path = os.path.relpath(r'..\demo_data\iea43_wra_data_model.json', cwd)
    with open(file_path) as json_file:
        meta_data = json.load(json_file)
    print("Loaded JSON file from local machine.")
except FileNotFoundError:
    # If the file isn't found, load it from GitHub
    url = "https://raw.githubusercontent.com/IEA-Task-43/digital_wra_data_standard/master/demo_data/iea43_wra_data_model.json"
    resp = requests.get(url)
    meta_data = json.loads(resp.text)
    print("Loaded JSON file from GitHub.")

### Show high level Data Model information

In [ ]:
print('Author name:\t\t\t{}'.format(meta_data['author']))
print('Author from:\t\t\t{}'.format(meta_data['organisation']))
print('Date produced:\t\t\t{}'.format(meta_data['date']))
print('IEA Data Model version\t\t{}'.format(meta_data['version']))

print('Plant name:\t\t\t{}'.format(meta_data['plant_name']))
print('Plant type:\t\t\t{}'.format(meta_data['plant_type']))


### Show all the measurement locations

In [ ]:
# print a table of the meas_locs parameters.
meas_locs = []
for meas_loc in meta_data['measurement_location']:
    meas_locs.append({
        'Name': meas_loc['name'],
        'Latitude [Decimal Degrees]': meas_loc['latitude_ddeg'],
        'Longitude [Decimal Degrees]': meas_loc['longitude_ddeg'],
        'Station Type': meas_loc['measurement_station_type_id'],
        'Notes': meas_loc['notes'],
        'Last Updated': meas_loc['update_at'],
        'Mast Type': meas_loc['mast_properties']['mast_geometry_id'],
        'Mast Height': meas_loc['mast_properties']['mast_height_m'],
        'Mast OEM': meas_loc['mast_properties']['mast_oem']
    })

meas_locs_df = pd.DataFrame(meas_locs)
meas_locs_df.set_index('Name', inplace=True)
display(meas_locs_df)

### Logger main configurations

In [ ]:
logger_main_config = []
for meas_loc in meta_data['measurement_location']:
    for log_config in meas_loc['logger_main_config']:
        logger_main_config.append(log_config)

logger_main_config_df = pd.DataFrame(logger_main_config)
display(logger_main_config_df.set_index('logger_name'))

### Some functions to pull out and format the measurement points set ups.

In [ ]:
def _flatten_sensor_dict(sensor):
    """
        Flatten the sensor dictionary retrieved from jason
        assigning all the sub-dictionaries to the main dictionary.

        :param sensor: The sensor dictionary retrieved for a single configuration
                           option and meas_point id.
        :type sensor: dict
        :return: output
        :rtype: dict

    """
    output = {key: value for key, value in sensor.items() if (type(value) != list) or (value == {})}
    for key, value in zip(sensor.keys(), sensor.values()):
        if (type(value) == list):
            if key == 'calibration':
                value = {key + "_" + k: v for k, v in value[0].items()}
            output.update(value)
    return output

In [ ]:
def rename_variables(input_dict, root_name):
    for var_to_rename in ['height_m', 'serial_number', 'update_at', 'notes']:
        if var_to_rename in list(input_dict.keys()):
            input_dict[ root_name + '_' + var_to_rename] = input_dict.pop(var_to_rename)
    return input_dict

In [ ]:
def replace_none_date(input_dict):
    for date_str in ['date_from', 'date_to']:
        if input_dict[date_str] is None:
            input_dict[date_str] = '2100-12-31'
    return input_dict

In [ ]:
def get_meas_points(meas_points):
    
    meas_points_flatten = []
    for meas_point in meas_points:
#         meas_point = _flatten_meas_point_dict(meas_point)
        log_meas_configs = sorted( meas_point['logger_measurement_config'], key=lambda i: i['date_from'])
        log_meas_configs = [replace_none_date(rename_variables(log_meas_config, 'log_meas_config')) for log_meas_config in log_meas_configs]
        sensors = [replace_none_date(rename_variables(_flatten_sensor_dict(sensor), 'sensor')) for sensor in meas_point['sensor']]
        if meas_point['mounting_arrangement'] is not None:
            mounting_arrangements = [replace_none_date(rename_variables(mntg_arrang, 'mounting_arrangement')) 
                                 for mntg_arrang in meas_point['mounting_arrangement']]
        else:
            mounting_arrangements = {}
        
        date_from = [log_meas_config['date_from'] for log_meas_config in log_meas_configs]
        date_to = [log_meas_config['date_to'] for log_meas_config in log_meas_configs]
        for sensor in sensors:
            date_from.append(sensor['date_from'])
            date_to.append(sensor['date_to'])
        for mntg_arrang in mounting_arrangements:
            date_from.append(mntg_arrang['date_from'])
            date_to.append(mntg_arrang['date_to'])
        
        date_from.extend(date_to)
        dates = np.unique(date_from)
        for i in range(len(dates)-1): 
            good_log_meas_config = {}
            for log_meas_config in log_meas_configs:
                if (log_meas_config['date_from'] <= dates[i]) & (log_meas_config['date_to'] > dates[i]):
                    good_log_meas_config = log_meas_config.copy()
            if good_log_meas_config != {}:
                for sensor in sensors: 
                    if (sensor['date_from'] <= dates[i]) & (sensor['date_to'] > dates[i]) :
                        good_log_meas_config.update(sensor)
                for mntg_arrang in mounting_arrangements:
                    if (mntg_arrang['date_from'] <= dates[i]) & (mntg_arrang['date_to'] > dates[i]) :
                        good_log_meas_config.update(mntg_arrang)
                good_log_meas_config['date_to'] = dates[i+1]
                good_log_meas_config['date_from'] = dates[i]
                good_log_meas_config.update(meas_point)
                del good_log_meas_config['logger_measurement_config']
                del good_log_meas_config['sensor'] 
                meas_points_flatten.append(good_log_meas_config)
    return meas_points_flatten 

In [ ]:
def _format_sensor_table(meas_points, table_type='full'):
    
    if table_type == 'full':
        header = ['name', 'measurement_units', 'oem',
                  'height_m', 'boom_orientation_deg', 'vane_dead_band_orientation_deg',
                  'date_from', 'date_to', 'connection_channel', 'log_meas_config_height_m', 'slope', 'offset', 'calibration_slope',
                  'calibration_offset']
        header_for_report = ['Instrument Name', 'Units', 'Sensor OEM',
                        'Height [m]', 'Boom Orient. [deg, mag N]', 'Dead Band Orient. [deg, mag N]',
                        'Date From', 'Date To', 'Logger Channel', 'Logger Stated Height [m]', 'Logger Slope', 'Logger Offset', 'Calibration Slope',
                        'Calibration Offset']
    elif table_type == 'meas_points':
        header = ['name', 'measurement_type_id', 'height_m', 'boom_orientation_deg']
        header_for_report = ['Instrument Name', 'Measurement Type', 'Height [m]', 'Boom Orient. [deg, mag N]']    
    elif table_type == 'speed_info':
        header = ['name', 'measurement_units', 'oem', 'model', 'sensor_serial_number',
                  'height_m', 'boom_orientation_deg', 
                  'date_from', 'date_to', 'slope', 'offset', 'calibration_slope',
                  'calibration_offset', 'measurement_type_id']
        header_for_report = ['Instrument Name', 'Units', 'Sensor Make', 'Sensor Model', 'Serial No',
                             'Height [m]', 'Boom Orient. [deg, mag N]',
                             'Date From', 'Date To', 'Logger Slope', 'Logger Offset', 'Calibration Slope',
                             'Calibration Offset', 'measurement_type_id']
    elif table_type == 'direction_info':
        header = ['name', 'measurement_units', 'oem', 'model', 'sensor_serial_number',
                  'height_m', 'boom_orientation_deg', 'vane_dead_band_orientation_deg', 
                  'date_from', 'date_to', 'offset', 'measurement_type_id']
        header_for_report = ['Instrument Name', 'Units', 'Sensor Make', 'Sensor Model', 'Serial No',
                             'Height [m]', 'Boom Orient. [deg, mag N]', 'Dead Band Orient. [deg, mag N]',
                             'Date From', 'Date To', 'Logger Offset', 'measurement_type_id']
    
    sensors_table_report = pd.DataFrame(meas_points)

    if any(elem not in sensors_table_report.columns for elem in header):
        ind_to_remove = [ind for ind, elem in enumerate(header) if elem not in sensors_table_report.columns]
        del header[ind_to_remove[0]]
        del header_for_report[ind_to_remove[0]]
    
    sensors_table_report = pd.DataFrame(sensors_table_report[header])
    if table_type == 'speed_info':
        sensors_table_report = sensors_table_report[sensors_table_report['measurement_type_id'] == 'wind_speed']
        del sensors_table_report['measurement_type_id']
    if table_type == 'direction_info':
        sensors_table_report = sensors_table_report[sensors_table_report['measurement_type_id'] == 'wind_direction']
        del sensors_table_report['measurement_type_id']
    
    if 'date_from' in sensors_table_report.columns:
        sensors_table_report['date_from'] = pd.to_datetime(sensors_table_report['date_from'].values.astype(str)).strftime("%d-%b-%Y")
    if 'date_to' in sensors_table_report.columns:
        sensors_table_report['date_to'] = pd.to_datetime(sensors_table_report['date_to'].values.astype(str)).strftime("%d-%b-%Y")

    sensors_table_report = sensors_table_report.replace({np.nan: '-', 'NaT': '-', '31-Dec-2100':'-'})
    sensors_table_report.rename(columns={k: h for k, h in zip(header, header_for_report)}, inplace=True)
    index_name = 'Instrument Name'
    sensors_table_report = sensors_table_report.set_index(index_name)
    
    return sensors_table_report

### The main measurment points

In [ ]:
for meas_loc in meta_data['measurement_location']:  
    logger_meas_configs = get_meas_points(meas_loc['measurement_point'])
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='meas_points')
    display(sensors_table.drop_duplicates())

### More details on each measurement point

In [ ]:
for meas_loc in meta_data['measurement_location']:
    logger_meas_configs = get_meas_points(meas_loc['measurement_point'])
    sensors_table = _format_sensor_table(logger_meas_configs)
    display(sensors_table)

### Anemometer (or wind speed) specific table

In [ ]:
for meas_loc in meta_data['measurement_location']:    
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='speed_info')
    display(sensors_table.drop_duplicates())

### Wind Vane (or wind direction) specific table

In [ ]:
for meas_loc in meta_data['measurement_location']:    
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='direction_info')
    display(sensors_table.drop_duplicates())

In [ ]:
logger_meas_configs